# LAB-21 直方圖及熱度圖

In [ ]:
import os
import sys
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## matlotlib 中文字型設定

In [ ]:
# 繪圖中文字型
# 設定中文字型
# 依不同平台 (Windows/Mac) 需設定不同中文字型

if sys.platform == "win32":
    # Windows 
    plt.rcParams['font.family'] = 'SimHei'
elif sys.platform == "darwin":
    plt.rcParams['font.family'] = 'Heiti TC' 
else:
    assert "未知作業系統"

## 讀取測試資料

In [ ]:
# 透過 LAB-18 產出的測試資料
csv_file = "test_data2.csv"
df = pd.read_csv(csv_file)

In [ ]:
df.head()

## 數值分布分析

In [ ]:
def plot_hist(data, bins_cnt):
    plt.hist(data, bins=bins_cnt, edgecolor='black', rwidth=0.75)             # bins 設定長條數量
    plt.title('交易延遲')
    plt.xlabel('回應時間')
    plt.ylabel('筆數')
    plt.show()

In [ ]:
# 快速統計 交易延遲的分布
# 繪製直方圖
plot_hist(df["交易延遲"], 20)


In [ ]:
# 針對 < 1000 的分析
df2 = df[df["交易延遲"] <= 1000]["交易延遲"]
plot_hist(df2, 20)

## 加上累計百分比

In [ ]:
bins = np.array([100,200,300,400,500,800,1000,2000,5000,100000000])
bin_labels = [ "100ms+","200ms+","300ms+","400ms+","500ms+","800ms+","> 1s+","> 2s","> 5s"]
#bin_labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins) - 1)]


In [ ]:
# 計算每個 bin 的頻率
hist, _ = np.histogram(df['交易延遲'], bins=bins)
df['bin'] = pd.cut(df['交易延遲'], bins=bins, labels=bin_labels, right=True)
bin_counts = df['bin'].value_counts().sort_index()

In [ ]:
# 計算累計百分比
total = bin_counts.sum()
cumulative_percent = (bin_counts.cumsum() / total * 100)

In [ ]:
# 創建圖表和第二軸
fig, ax1 = plt.subplots(figsize=(12, 7))
ax2 = ax1.twinx()

# 繪製等寬的長條圖
bar_width = 0.8
bar_positions = np.arange(len(bin_labels))
bars = ax1.bar(bar_positions, bin_counts, width=bar_width, color='skyblue', edgecolor='black')

# 繪製累計百分比折線圖
line = ax2.plot(bar_positions, cumulative_percent, 'ro-', linewidth=2, markersize=8)

# 設置 X 軸標籤
ax1.set_xticks(bar_positions)
ax1.set_xticklabels(bin_labels)

# 在長條上添加頻率數值
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 5,
            f'{height:.0f}', ha='center', va='bottom')

# 在折線圖點上添加百分比標籤
for i, value in enumerate(cumulative_percent):
    ax2.text(bar_positions[i], value + 2, f'{value:.2f}%', ha='center')

# 設置圖表標題和軸標籤
ax1.set_title('交易延遲')
ax1.set_xlabel('回應時間')
ax1.set_ylabel('筆數')
ax2.set_ylabel('累計百分比 (%)')

# 設置 Y 軸範圍
ax2.set_ylim(0, 105)

# 添加圖例
ax1.legend(['筆數'], loc='upper left')
#ax2.legend(['累計百分比'], loc='lower right')

# 添加網格線
ax1.grid(axis='y', linestyle='--', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 針對 > 5000 的分析
df3 = df[df["交易延遲"] > 5000].copy()
plot_hist(df3["交易延遲"], 20)

In [ ]:
df3["交易延遲"].describe()

In [ ]:
df3.head()

## 熱度圖

In [ ]:
# Ｘ軸 交易代碼
# Y軸 交易主機
# 計算每個 X-Y 組合的統計數據
# 這裡使用 count 來計算筆數
pivot_table = df3.pivot_table(index='交易主機', columns='交易代碼', values='交易延遲', aggfunc='count')

# 繪製熱度圖
#plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': '筆數'})
plt.title('交易代碼 vs 交易主機 筆數統計熱度圖')
plt.tight_layout()
plt.show()


In [ ]:
# 將異常交易，另存新檔檢視或查核
csvf = "lab-21.csv"
df3.to_csv(csvf, index=False, encoding='utf-8-sig')

In [ ]:
## 依交易時間，畫熱度圖
df["hour"] = df["交易時間"].apply(lambda x : x[11:13])

In [ ]:
# Ｘ軸 hour
# Y軸 交易代碼
# 計算每個 X-Y 組合的統計數據
# 這裡使用 count 來計算筆數
pivot_table = df.pivot_table(index='交易主機', columns='hour', values='交易延遲', aggfunc='count')

# 繪製熱度圖
#plt.figure(figsize=(10, 8))
sns.heatmap(pivot_table, annot=False, fmt='d', cmap='YlOrRd', cbar_kws={'label': '筆數'})
plt.title('交易時間 vs 交易主機 筆數統計熱度圖')
plt.tight_layout()
plt.show()